### 1. 安装环境

- 系统：centos8
- JDK：1.8_231
- hadoop: 3.2.1

### 2. 主机准备

- 主机名： hadoop
- vi /etc/hostname
- 主机名与IP地址映射： /etc/hosts


In [ ]:
    # vi /etc/hosts
    
    IP地址   hadoop

- 防火墙关闭


In [ ]:
    # systemctl status firewalld

- 创建hadoop用户，命令如下：

In [ ]:
    # useradd hadoop
    # passwd hadoop 
    
    输入2遍【hadoop】

- hadoop用户环境变量，配置JDK； /home/hadoop/.bash_profile



### 3. 安装hadoop

- 配置hadoop用户的免密码登录（ssh）



In [ ]:
    $ cd ~
    
    $ ssh-keygen -t rsa  [一路回车]
    Generating public/private rsa key pair.
    Enter file in which to save the key (/home/hadoop/.ssh/id_rsa): 
    Enter passphrase (empty for no passphrase): 
    Enter same passphrase again: 
    Your identification has been saved in /home/hadoop/.ssh/id_rsa.
    Your public key has been saved in /home/hadoop/.ssh/id_rsa.pub.
    The key fingerprint is:
    SHA256:rHZD+W4NlnaUpIxIQ0Z4BQopVRv+8R7plMZNysmiOUg hadoop@hadoop
    The key's randomart image is:
    +---[RSA 2048]----+
    | .ooo+=o.        |
    |. .o.=+     .    |
    | .  +o.o o.o .   |
    |     ..B.Oo o    |
    |  E   o S .o     |
    | . . o O o= .    |
    |  . + o =o.+     |
    |     o . o. .    |
    |         ..      |
    +----[SHA256]-----+
    
    $ cd .ssh
    $ cat id_rsa.pub >> authorized_keys
    $ chmod 600 authorized_keys
    $ chmod 700 ~/.ssh

- 验证面密登录配置是否成功，命令：

In [ ]:
    $ ssh hadoop
    
    $ exit
    

- 上传安装文件到指定目录，/home/hadoop/tools

- 解压tar.gz文件，并移动解压后的目录到指定位置，/usr/, 命令如下：

In [ ]:
    $ tar zxvf hadoop-3.2.1.tar.gz
    
    $ ll
    
    总用量 540384
    drwxr-xr-x. 9 hadoop hadoop       149 9月  11 2019 hadoop-3.2.1
    -rwxrwxr-x. 1 hadoop hadoop 359196911 7月   7 11:19 hadoop-3.2.1.tar.gz
        
    $ exit
    
    # mv /home/hadoop/tools/hadoop-3.2.1 /usr/
    
    $ su - hadoop
    

- hadoop环境变量配置，~/.bash_profile, 命令如下：


In [ ]:
    $ vi ~/.bash_profile
    
    JAVA_HOME=/usr/jdk1.8.0_231
    HADOOP_HOME=/usr/hadoop-3.2.1
    PATH=$HADOOP_HOME/bin:$HADOOP_HOME/sbin:$JAVA_HOME/bin:$PATH

    export JAVA_HOME
    export HADOOP_HOME
    export PATH
    
    $ source .bash_profile

- hadoop的基本配置文件，hadoop-env.sh, 具体配置如下：

In [ ]:
    $ cd /usr/hadoop-3.2.1/etc/hadoop
    
    $ vi hadoop-env.sh
    
    54 export JAVA_HOME=/usr/jdk1.8.0_231

- 测试基本配置是否完成，命令如下：

In [ ]:
    $ hadoop version
    Hadoop 3.2.1
    Source code repository https://gitbox.apache.org/repos/asf/hadoop.git -r b3cbbb467e22ea829b3808f4b7b01d07e0bf3842
    Compiled by rohithsharmaks on 2019-09-10T15:56Z
    Compiled with protoc 2.5.0
    From source with checksum 776eaf9eee9c0ffc370bcbc1888737
    This command was run using /usr/hadoop-3.2.1/share/hadoop/common/hadoop-common-3.2.1.jar

- 准备目录，/usr/local/hadoop,用于job执行的临时文件目录，和数据存储


In [ ]:
    # cd /usr/local
    # mkdir hadoop
    # chown -R hadoop:hadoop hadoop

- 核心配置（1）core-site.xml, $HADOOP_HOME/etc/hadoop/core-site.xml, 具体配置内容：


In [ ]:
    $ vi /usr/hadoop-3.2.1/etc/hadoop/core-site.xml 
    
    <configuration>
    <!-- 指定HDFS路径地址，hadoop是与IP映射的主机名，9000是端口号 -->
    <property>
        <name>fs.defaultFS</name>
        <value>hdfs://hadoop:9000</value>
    </property>

    <!-- 指定hadoop运行过程中产生的文件存储目录 -->
    <property>
        <name>hadoop.tmp.dir</name>
        <value>/usr/local/hadoop/tmp</value>
    </property>
    </configuration>

- 核心配置（2）hdfs-site.xml,$HADOOP_HOME/etc/hadoop目录下，具体配置：

In [ ]:
<!-- 文件存储在hdfs上的副本数量-->
<property>
    <name>dfs.replication</name>
    <value>1</value>
</property>
<!-- hdfs web监听端口-->
<property>
    <name>dfs.namenode.http-address</name>
    <value>hadoop:9870</value>
</property>


<!-- namenode数据存储路径 -->
<property>
    <name>dfs.namenode.name.dir</name>
    <value>/usr/local/hadoop/dfs/name</value>
</property>

<!-- datanode数据存储路径 -->
<property>
    <name>dfs.datanode.data.dir</name>
    <value>/usr/local/hadoop/dfs/data</value>
</property>

- 核心配置（3）mapred-site.xml,具体配置：

In [ ]:
<property>
    <name>mapreduce.framework.name</name>
    <value>yarn</value>
</property>

<property>
    <name>mapreduce.application.classpath</name>
    <value>
    /usr/hadoop-3.2.1/etc/hadoop:/usr/hadoop-3.2.1/share/hadoop/common/lib/*:/usr/hadoop-3.2.1/share/hadoop/common/*:/usr/hadoop-3.2.1/share/hadoop/hdfs:/usr/hadoop-3.2.1/share/hadoop/hdfs/lib/*:/usr/hadoop-3.2.1/share/hadoop/hdfs/*:/usr/hadoop-3.2.1/share/hadoop/mapreduce/lib/*:/usr/hadoop-3.2.1/share/hadoop/mapreduce/*:/usr/hadoop-3.2.1/share/hadoop/yarn:/usr/hadoop-3.2.1/share/hadoop/yarn/lib/*:/usr/hadoop-3.2.1/share/hadoop/yarn/*
    </value>
</property>

- 核心配置（4）yarn-site.xml, 具体配置：

In [ ]:
<property>
    <name>yarn.resourcemanager.hostname</name>
    <value>hadoop</value>
</property>
<property>
    <name>yarn.nodemanager.aux-services</name>
    <value>mapreduce_shuffle</value>
</property>
<property>
    <name>mapreduce.application.classpath</name>
    <value>
    /usr/hadoop-3.2.1/etc/hadoop:/usr/hadoop-3.2.1/share/hadoop/common/lib/*:/usr/hadoop-3.2.1/share/hadoop/common/*:/usr/hadoop-3.2.1/share/hadoop/hdfs:/usr/hadoop-3.2.1/share/hadoop/hdfs/lib/*:/usr/hadoop-3.2.1/share/hadoop/hdfs/*:/usr/hadoop-3.2.1/share/hadoop/mapreduce/lib/*:/usr/hadoop-3.2.1/share/hadoop/mapreduce/*:/usr/hadoop-3.2.1/share/hadoop/yarn:/usr/hadoop-3.2.1/share/hadoop/yarn/lib/*:/usr/hadoop-3.2.1/share/hadoop/yarn/*
    </value>
</property>

- 核心配置（5）workers，具体配置：

In [ ]:
    hadoop

### 4. 启动hadoop集群（伪分布式）

- 首次启动hadoop，必须namenode格式化，命令如下：

In [ ]:
    $ hdfs namenode -format
    
    2020-07-07 14:25:59,234 INFO common.Storage: Storage directory /usr/local/hadoop/dfs/name has been successfully formatted.

- 启动两个服务（HDFS、yarn），命令如下：


In [ ]:
    $ start-dfs.sh
    
    $ start-yarn.sh
    
    $ jps
    
    14337 Jps
    11608 jar
    13610 SecondaryNameNode
    13867 ResourceManager
    13388 DataNode
    13261 NameNode
    13997 NodeManager
    

- HDFS管理命令

    1. hdfs安全模式查看
    

In [ ]:
        $ hdfs dfsadmin -safemode get
    
        Safe mode is OFF

        注意：Safe mode is OFF，说明HDFS安全模式已经关闭，实现对数据的读写操作

    2. 查看根目录结构，命令如下：

In [ ]:
    $ hdfs dfs -ls /

    3. 创建目录，命令如下：
    

In [ ]:
    $ hdfs dfs -mkdir /data
    
    $ hdfs dfs -ls /

    4. 递归创建目录，命令如下：

In [ ]:
    $ hdfs dfs -mkdir -p /data/subdata/input
    
    $ hdfs dfs -ls -R /

    5. 上传本地文件到hdfs指定目录，命令如下：

In [ ]:
    $ hdfs dfs -put jdk-8u231-linux-x64.tar.gz /data/
    
    $ hdfs dfs -ls /data
    

    6. 下载hdfs数据文件到本地操作系统，命令如下：
    

In [ ]:
    $ hdfs dfs -get /data/jdk-8u231-linux-x64.tar.gz ./
    
    $ ll
    

    7. 复制文件，命令如下：

In [ ]:
    $ hdfs dfs -cp /data/jdk-8u231-linux-x64.tar.gz /data/subdata/jdk.tar.gz
    

    8. 删除文件，命令如下：
    

In [ ]:
    $ hdfs dfs -rm -r /data/subdata/

### 5. HDFS管理命令

- 安全模式




In [ ]:
    $ hdfs dfsadmin -safemode get
    

    hdfs dfsadmin -safemode get|enter|leave|wait

- report命令

In [ ]:
    $ hdfs dfsadmin -report
    
    

In [ ]:
Configured Capacity: 37558423552 (34.98 GB)
Present Capacity: 29635686400 (27.60 GB)
DFS Remaining: 29440000000 (27.42 GB)
DFS Used: 195686400 (186.62 MB)
DFS Used%: 0.66%
Replicated Blocks:
        Under replicated blocks: 0
        Blocks with corrupt replicas: 0
        Missing blocks: 0
        Missing blocks (with replication factor 1): 0
        Low redundancy blocks with highest priority to recover: 0
        Pending deletion blocks: 0
Erasure Coded Block Groups: 
        Low redundancy block groups: 0
        Block groups with corrupt internal blocks: 0
        Missing block groups: 0
        Low redundancy blocks with highest priority to recover: 0
        Pending deletion blocks: 0

-------------------------------------------------
Live datanodes (1):

Name: 192.168.1.103:9866 (hadoop)
Hostname: hadoop
Decommission Status : Normal
Configured Capacity: 37558423552 (34.98 GB)
DFS Used: 195686400 (186.62 MB)
Non DFS Used: 7922737152 (7.38 GB)
DFS Remaining: 29440000000 (27.42 GB)
DFS Used%: 0.52%
DFS Remaining%: 78.38%
Configured Cache Capacity: 0 (0 B)
Cache Used: 0 (0 B)
Cache Remaining: 0 (0 B)
Cache Used%: 100.00%
Cache Remaining%: 0.00%
Xceivers: 1
Last contact: Tue Jul 07 14:54:09 CST 2020
Last Block Report: Tue Jul 07 14:28:45 CST 2020
Num of Blocks: 2
